In [1]:
import os, re, glob
# import xml.etree.ElementTree as ET
from ipymarkup import show_box_markup
from ipymarkup.palette import palette, BLUE, RED, GREEN

In [2]:
PATH_DATA = '/Users/a19028558/Desktop/sber_NLP_learning/RuREBus/train_data/'

In [3]:
types = ('*.txt', '*.ann') # the tuple of file types
files_grabbed = []
for files in types:
    files_grabbed.extend(glob.glob(PATH_DATA +'**/'+ files , recursive=True))
# glob.glob(PATH_DATA + '**/*.txt' , recursive=True)
files_grabbed[:1]

['/Users/a19028558/Desktop/sber_NLP_learning/RuREBus/train_data/train_part_1/20336241021100524345002_22_part_1.txt']

In [4]:
def read_text_with_markup(text_file_name, markup_file_name):
    with open(text_file_name, "r") as r:
            text = r.read()
    spans = []
    for line in open(markup_file_name):
        line_list = re.split('[\s,\t]', line)
        span = line_list[1]
        start_pos = line_list[2]
        end_pos = line_list[3]
        try:
            spans.append((int(start_pos), int(end_pos), str(span)))
        except:
            pass
    return text, spans



In [5]:
# sorted(files_grabbed)

In [6]:
data = []
i = 0
sorted_sample_name = sorted(files_grabbed)
data = []
while i < len(sorted_sample_name) - 1:
    sample_name_ann = sorted_sample_name[i]
    sample_name_txt = sorted_sample_name[i+1]
    if i % 2 == 0 and i != 0:
        data.append(read_text_with_markup(sample_name_txt, sample_name_ann))
    i += 1

In [7]:
# len(sorted_sample_name)

In [8]:
# data[175][0]

In [9]:
show_box_markup(data[175][0], data[175][1], palette=palette(ACT=BLUE, CMP=GREEN, MET=RED))

In [10]:
from enum import IntEnum
from collections import namedtuple


from razdel import tokenize

class Label(IntEnum):
    OUTER = 0
    BEGIN = 1
    INNER = 2
    
Sample = namedtuple("Sample", "text,tokens,spans,labels")

def text_span_to_sample(text, spans):
    labels = []
    tokens = list(tokenize(text))
    for token in tokens:
        label = Label.OUTER
        for span in spans:
            span_begin, span_end, tag = span
            if token.start == span_begin:
                label = Label.BEGIN
            elif token.start > span_begin and token.stop <= span_end:
                label = Label.INNER
        labels.append(label)
    # Проверяем инвариант отсутствия последовательных пар (0, 2)
    assert len([1 for i in range(len(labels) - 1) if (labels[i], labels[i+1]) == (0, 2)]) == 0
    return Sample(text, tokens, spans, labels)

samples = []
for text, spans in data:
    samples.append(text_span_to_sample(text, spans))

show_box_markup(samples[0].text, samples[0].spans, palette=palette(INST=BLUE, BIN=RED, ECO=GREEN))
print(samples[0].labels)
print(len(samples))

[<Label.OUTER: 0>, <Label.OUTER: 0>, <Label.BEGIN: 1>, <Label.INNER: 2>, <Label.INNER: 2>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.BEGIN: 1>, <Label.INNER: 2>, <Label.INNER: 2>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.BEGIN: 1>, <Label.INNER: 2>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.BEGIN: 1>, <Label.INNER: 2>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.BEGIN: 1>, <Label.INNER: 2>, <Label.INNER: 2>, <Label.INNER: 2>, <Label.OUTER: 0>, <Label.OUTER: 0>, <Label.OU

In [11]:
import random
random.shuffle(samples)

sample_len = len(samples)
perc70 = sample_len * 0.7
perc85 = sample_len * 0.85
train = samples[:int(perc70)]
val = samples[int(perc70):int(perc85)]
test = samples[int(perc85):]
print(len(train), len(val), len(test))

130 28 29


In [12]:
char_set = ["<pad>", "<unk>"] + list({ch for sample in train for token in sample.tokens for ch in token.text})
print(char_set)

['<pad>', '<unk>', 'ц', 'u', 'm', '"', 'o', 'к', 'M', 'K', 'I', '»', 'с', 'Е', 'Д', 'п', 'Ф', 'Т', 'ч', 'Ц', 'a', 'ф', 'V', ';', 'h', 'в', 'Щ', '4', '*', 'Х', 'С', 'l', 'Э', '=', 'е', '_', '£', '┘', '│', '─', 'w', 's', 'т', '5', '>', 'H', '0', 'Л', ']', 'X', 'ш', 'Г', 'д', 'җ', 'О', 'd', 'Ч', 'y', 'ь', '«', 'Р', '├', 'T', '{', 'щ', 'ё', ':', 'ю', 'e', 'k', 'н', 'Ж', 'Ь', '┤', 'g', 'Ъ', 'я', '<', 'O', 'б', 'ъ', 'E', 'г', 'v', 'C', '/', 'К', '!', 'М', '1', '7', 'G', 'Ы', 'З', '.', 'i', '}', '”', 'n', '+', 'Ш', 'ж', ',', 'A', '@', 'И', '“', '(', 'D', 'Й', 'А', '3', 'F', ')', 'R', 'І', 'у', '┌', 'л', '–', 'f', 'P', 'r', 'о', 'П', 'c', 'У', 'Н', 'р', 'Ю', 'z', 'b', 'Ә', 'й', 'м', '2', 'Я', '—', '№', 'В', 'B', '┐', 'а', 'Б', 'p', '%', '×', 'W', '9', 'S', 'x', '-', '\uf0b7', 'N', '[', 'Z', 'э', 'L', 't', '∑', '6', '…', 'ы', 'х', 'Σ', 'j', 'и', 'Ё', '■', 'ә', '8', 'з', 'U', '└']


In [13]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, RandomSampler

class NerCharsDataset(Dataset):
    def __init__(self, samples, char_set, max_seq_len=500, max_char_seq_len=50):
        assert len(samples) != 0
        self.samples = []
        self.tokens = []
        self.texts = []
        for sample in samples:
            inputs = torch.zeros((max_seq_len, max_char_seq_len), dtype=torch.long)
            for token_num, token in enumerate(sample.tokens[:max_seq_len]):
                for ch_num, ch in enumerate(token.text[:max_char_seq_len]):
                    char_index = char_set.index(ch) if ch in char_set else char_set.index("<unk>")
                    inputs[token_num][ch_num] = char_index
            labels = torch.zeros((max_seq_len,), dtype=torch.long)
            input_labels = [int(i) for i in sample.labels[:max_seq_len]]
            labels[:len(input_labels)] = torch.LongTensor(input_labels)
            self.samples.append((torch.LongTensor(inputs), torch.LongTensor(labels)))
            self.tokens.append(sample.tokens[:max_seq_len])
            self.texts.append(sample.text)
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        return self.samples[index]

    def get_tokens(self, index):
        return self.tokens[index]
    
    def get_text(self, index):
        return self.texts[index]


BATCH_SIZE = 32

train_data = NerCharsDataset(train, char_set)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, sampler=train_sampler)

val_data = NerCharsDataset(val, char_set)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

test_data = NerCharsDataset(test, char_set)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [14]:
print(test_data[0])
for sample in train_loader:
    inputs, labels = sample
    print(inputs.size())
    print(labels.size())
    break

(tensor([[126,  42,  25,  ...,   0,   0,   0],
        [124, 123,  12,  ...,   0,   0,   0],
        [110,  52, 134,  ...,   0,   0,   0],
        ...,
        [ 17, 142,   7,  ...,   0,   0,   0],
        [ 42, 128, 116,  ...,   0,   0,   0],
        [ 15, 123,   0,  ...,   0,   0,   0]]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 0, 1,
        0, 1, 2, 1, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 0, 1, 2, 2, 2, 2,
        0, 1, 2, 1, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 2, 0, 0, 0, 1, 2, 0, 1, 2, 2, 2, 0, 0, 0,
        0, 0, 0, 1, 2, 2, 0, 1, 1, 1, 2, 2, 0, 1, 1, 2, 2, 0, 1, 2, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 2, 1, 0, 0, 0, 1, 1, 0, 0, 1, 2, 0, 1, 2, 0, 

In [15]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule
from pytorch_lightning.metrics import Accuracy
 

class TemplateModel(LightningModule):
    def __init__(self):
        super().__init__()
        
        self.loss = nn.CrossEntropyLoss()
        self.valid_accuracy = Accuracy()
        self.test_accuracy = Accuracy()
    
    def forward(self, inputs, labels):
        raise NotImplementedError("forward not implemented")
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return [optimizer]
    
    def training_step(self, batch, _):
        inputs, labels = batch
        loss, logits = self(inputs, labels)
        return loss
    
    def validation_step(self, batch, _):
        inputs, labels = batch
        val_loss, logits = self(inputs, labels)
        self.valid_accuracy.update(logits, labels)
        self.log("val_loss", val_loss, prog_bar=True)
        self.log("val_acc", self.valid_accuracy)

    def validation_epoch_end(self, outs):
        self.log("val_acc_epoch", self.valid_accuracy.compute(), prog_bar=True)

    def test_step(self, batch, _):
        inputs, labels = batch
        test_loss, logits = self(inputs, labels)
        self.test_accuracy.update(logits, labels)
        self.log("test_loss", test_loss, prog_bar=True)
        self.log("test_acc", self.test_accuracy)

    def test_epoch_end(self, outs):
        self.log("test_acc_epoch", self.test_accuracy.compute(), prog_bar=True)

In [16]:
import torch
from torch import nn
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping

class SuperSimpleModel(TemplateModel):
    def __init__(self, char_set_size, char_embedding_dim=32, classes_count=3, char_max_seq_len=50):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim) # а -> [0.1, ... 0.3]
        self.out_layer = nn.Linear(char_max_seq_len * char_embedding_dim, classes_count)

    def forward(self, inputs, labels):
        projections = self.embeddings_layer.forward(inputs)
        projections = projections.reshape(projections.size(0), projections.size(1), -1) # <-
        logits = self.out_layer.forward(projections)
        logits = logits.transpose(1, 2)
        loss = self.loss(logits, labels)
        return loss, logits


super_simple_model = SuperSimpleModel(len(char_set))
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=5,
    verbose=True,
    mode="min" 
)
trainer = Trainer(
    gpus=0,
    checkpoint_callback=False,
    accumulate_grad_batches=1,
    max_epochs=150,
    progress_bar_refresh_rate=10,
    callbacks=[early_stop_callback])
trainer.fit(super_simple_model, train_loader, val_loader)
trainer.test(super_simple_model, test_loader)

/Users/a19028558/Desktop/sber_NLP_learning/fastText-0.9.2/newvenv/lib/python3.7/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `Accuracy` was deprecated since v1.3.0 in favor of `torchmetrics.classification.accuracy.Accuracy`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name             | Type             | Params
------------------------------------------------------
0 | loss             | CrossEntropyLoss | 0     
1 | valid_accuracy   | Accuracy         | 0     
2 | test_accuracy    | Accuracy         | 0     
3 | embeddings_layer | Embedding        | 5.6 K 
4 | out_layer        | Linear           | 4.8 K 
------------------------------------------------------
10.4 K    Trainable params
0         Non-trainable params
10.4 K    Total params
0.041     Total estimated model params size (MB)


Epoch 0:   0%|                                                                                                | 0/6 [00:00<?, ?it/s]

/Users/a19028558/Desktop/sber_NLP_learning/fastText-0.9.2/newvenv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:103: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f'The dataloader, {name}, does not have many workers which may be a bottleneck.'
/Users/a19028558/Desktop/sber_NLP_learning/fastText-0.9.2/newvenv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:103: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f'The dataloader, {name}, does not have many workers which may be a bottleneck.'


Epoch 0: 100%|████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.76it/s]
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                                             | 0/1 [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.936


...

Epoch 25: 100%|████████████████████████████| 6/6 [00:03<00:00,  1.80it/s, loss=0.697, v_num=18, val_loss=0.716, val_acc_epoch=0.682]
Validating: 0it [00:00, ?it/s]
Epoch 26: 100%|████████████████████████████| 6/6 [00:03<00:00,  1.80it/s, loss=0.696, v_num=18, val_loss=0.719, val_acc_epoch=0.686]
Validating: 0it [00:00, ?it/s]
Epoch 27: 100%|████████████████████████████| 6/6 [00:03<00:00,  1.77it/s, loss=0.689, v_num=18, val_loss=0.720, val_acc_epoch=0.685]
Validating: 0it [00:00, ?it/s]
Epoch 28: 100%|████████████████████████████| 6/6 [00:03<00:00,  1.71it/s, loss=0.703, v_num=18, val_loss=0.739, val_acc_epoch=0.674]
Validating: 0it [00:00, ?it/s]
Epoch 29: 100%|████████████████████████████| 6/6 [00:03<00:00,  1.75it/s, loss=0.694, v_num=18, val_loss=0.765, val_acc_epoch=0.676]
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                                             | 0/1 [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 5 records. Best score: 0.716. Signaling Trainer to stop.


Testing: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 19.04it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.6932413578033447,
 'test_acc_epoch': 0.6932413578033447,
 'test_loss': 0.6971759796142578}
--------------------------------------------------------------------------------


/Users/a19028558/Desktop/sber_NLP_learning/fastText-0.9.2/newvenv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:103: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f'The dataloader, {name}, does not have many workers which may be a bottleneck.'


[{'test_loss': 0.6971759796142578,
  'test_acc': 0.6932413578033447,
  'test_acc_epoch': 0.6932413578033447}]

## Бесконтекстная модель

In [17]:
import torch
from torch import nn
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping

class SuperSimpleModel(TemplateModel):
    def __init__(self, char_set_size, char_embedding_dim=32, classes_count=3, char_max_seq_len=50):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim) # а -> [0.1, ... 0.3]
        self.out_layer = nn.Linear(char_max_seq_len * char_embedding_dim, classes_count)

    def forward(self, inputs, labels):
        projections = self.embeddings_layer.forward(inputs)
        projections = projections.reshape(projections.size(0), projections.size(1), -1) # <-
        logits = self.out_layer.forward(projections)
        logits = logits.transpose(1, 2)
        loss = self.loss(logits, labels)
        return loss, logits


super_simple_model = SuperSimpleModel(len(char_set))
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=5,
    verbose=True,
    mode="min" 
)
trainer = Trainer(
    gpus=0,
    checkpoint_callback=False,
    accumulate_grad_batches=1,
    max_epochs=150,
    progress_bar_refresh_rate=10,
    callbacks=[early_stop_callback])
trainer.fit(super_simple_model, train_loader, val_loader)
trainer.test(super_simple_model, test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name             | Type             | Params
------------------------------------------------------
0 | loss             | CrossEntropyLoss | 0     
1 | valid_accuracy   | Accuracy         | 0     
2 | test_accuracy    | Accuracy         | 0     
3 | embeddings_layer | Embedding        | 5.6 K 
4 | out_layer        | Linear           | 4.8 K 
------------------------------------------------------
10.4 K    Trainable params
0         Non-trainable params
10.4 K    Total params
0.041     Total estimated model params size (MB)


Epoch 0: 100%|████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.65it/s]
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                                             | 0/1 [00:00<?, ?it/s]

Metric val_loss improved. New best score: 1.075


Epoch 1: 100%|██████████████████████████████| 6/6 [00:03<00:00,  1.65it/s, loss=1.17, v_num=19, val_loss=1.080, val_acc_epoch=0.643]
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                                             | 0/1 [00:00<?, ?it/s]

Metric val_loss improved by 0.232 >= min_delta = 0.0. New best score: 0.843


...

Metric val_loss improved by 0.001 >= min_delta = 0.0. New best score: 0.714


Epoch 28: 100%|████████████████████████████| 6/6 [00:03<00:00,  1.60it/s, loss=0.706, v_num=19, val_loss=0.714, val_acc_epoch=0.687]
Validating: 0it [00:00, ?it/s]
Epoch 29: 100%|████████████████████████████| 6/6 [00:03<00:00,  1.65it/s, loss=0.701, v_num=19, val_loss=0.721, val_acc_epoch=0.675]
Validating: 0it [00:00, ?it/s]
Epoch 30: 100%|████████████████████████████| 6/6 [00:03<00:00,  1.63it/s, loss=0.694, v_num=19, val_loss=0.715, val_acc_epoch=0.689]
Validating: 0it [00:00, ?it/s]
Epoch 31: 100%|████████████████████████████| 6/6 [00:03<00:00,  1.66it/s, loss=0.699, v_num=19, val_loss=0.732, val_acc_epoch=0.673]
Validating: 0it [00:00, ?it/s]
Epoch 32: 100%|████████████████████████████| 6/6 [00:03<00:00,  1.63it/s, loss=0.698, v_num=19, val_loss=0.728, val_acc_epoch=0.682]
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                                             | 0/1 [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 5 records. Best score: 0.714. Signaling Trainer to stop.


Testing: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16.93it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.707793116569519,
 'test_acc_epoch': 0.707793116569519,
 'test_loss': 0.667972207069397}
--------------------------------------------------------------------------------


[{'test_loss': 0.667972207069397,
  'test_acc': 0.707793116569519,
  'test_acc_epoch': 0.707793116569519}]

## Метрики

Можно использовать как классические мультиклассификационнные метрики, так и метрики специально для NER.

Например, число точных и частичных совпадений спанов, пропущенных и лишних спанов.

In [22]:
def get_spans(labels, tokens):
    spans = []
    for i, (label, token) in enumerate(zip(labels, tokens)):
        if label == 1:
            spans.append((token.start, token.stop, "INST"))
        elif label == 2:
            assert len(spans) != 0, "Incorrect label sequence: {}".format(labels)
            old_begin, _, old_tag = spans[-1]
            spans[-1] = (old_begin, token.stop, old_tag)
    return spans


def compare_span_sets(left_spans, right_spans):
    exact, partial, missing = 0, 0, 0
    for left_span in left_spans:
        is_missing = True
        for right_span in right_spans:
            if left_span == right_span:
                exact += 1
                is_missing = False
                break
            ls, le, _ = left_span
            rs, re, _ = right_span
            # [ls le] [rs re]
            # [rs re] [ls le]
            if not (ls <= le <= rs <= re or rs <= re <= ls <= le):
                is_missing = False
                partial += 1
                break            
        if is_missing:
            missing += 1
    return exact, partial, missing


def calc_metrics(true_labels, predicted_labels, tokens):
    # Метрики классификации
    one_tp, one_fp, one_fn = 0, 0, 0
    for true, predicted in zip(true_labels, predicted_labels):
        for l1, l2 in zip(true, predicted):
            if l1 == 1 and l2 == 1:
                one_tp += 1
            elif l1 != 1 and l2 == 1:
                one_fp += 1
            elif l1 == 1 and l2 !=1:
                one_fn += 1
    if one_tp + one_fp == 0:
        print("No positives!")
    else:
        print("1 Precision: {}, 1 Recall: {}".format(float(one_tp)/(one_tp + one_fp), float(one_tp)/(one_tp + one_fn)))

    # Специализированные метрики
    e, p, m, s = 0, 0, 0, 0
    for (true, predicted), sample_tokens in zip(zip(true_labels, predicted_labels), tokens):
        true_spans = get_spans(true, sample_tokens)
        predicted_spans = get_spans(predicted, sample_tokens)
        exact, partial, missing = compare_span_sets(true_spans, predicted_spans)
        _, _, spurius = compare_span_sets(predicted_spans, true_spans)
        e += exact
        p += partial
        m += missing
        s += spurius
    print("Exact: {}, partial: {}, missing: {}, spurius: {}".format(e, p, m, s))
            


def predict(model, test_loader, show_sample_index=2):
    model.eval()
    all_true_labels, all_predicted_labels, all_tokens, all_texts = [], [], [], []
    for batch_index, batch in enumerate(test_loader):
        inputs, true_labels = batch
        batch_size = inputs.size(0)
        _, logits = model(inputs.to("cpu"), true_labels.to("cpu"))
        predicted_labels = logits.max(dim=1)[1].detach().cpu()

        # Убираем неконсистентность BIO
        for sample_num, sample in enumerate(predicted_labels):
            for token_num, label in enumerate(sample):
                if token_num == 0 and label == 2:
                    predicted_labels[sample_num][0] = 1
                    continue
                prev_label = sample[token_num - 1]
                if label == 2 and prev_label == 0:
                    predicted_labels[sample_num][token_num] = 1

        all_true_labels.extend(true_labels)
        all_predicted_labels.extend(predicted_labels)
        for i in range(batch_size):
            all_tokens.append(test_data.get_tokens(batch_index * batch_size + i))
            all_texts.append(test_data.get_text(batch_index * batch_size + i))

    calc_metrics(all_true_labels, all_predicted_labels, all_tokens)
    print("PREDICTED:")
    show_box_markup(all_texts[show_sample_index],
                    get_spans(all_predicted_labels[show_sample_index], all_tokens[show_sample_index]),
                    palette=palette(PER=BLUE, ORG=RED, LOC=GREEN))

In [19]:
predict(super_simple_model, test_loader)

1 Precision: 0.3917841814837523, 1 Recall: 0.6240234375
Exact: 745, partial: 1021, missing: 282, spurius: 1016
PREDICTED:


## Контекстная модель: LSTM над конкатенацией

In [23]:
import torch
from torch import nn
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping

class LstmModel(TemplateModel):
    def __init__(self, char_set_size, char_embedding_dim=4, classes_count=3,
                 lstm_embedding_dim=8, char_max_seq_len=50):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim)
        self.dropout = nn.Dropout(0.4)
        self.lstm_layer = nn.LSTM(char_embedding_dim * char_max_seq_len, lstm_embedding_dim // 2,
                                  batch_first=True, bidirectional=True)
        self.out_layer = nn.Linear(lstm_embedding_dim, classes_count)

    def forward(self, inputs, labels):
        batch_size = inputs.size(0)
        seq_len = inputs.size(1)
        projections = self.embeddings_layer.forward(inputs)
        projections = projections.reshape(projections.size(0), projections.size(1), -1)
        output, _= self.lstm_layer(projections)
        output = self.dropout(output)
        logits = self.out_layer.forward(output)
        logits = logits.transpose(1, 2)
        loss = self.loss(logits, labels)
        return loss, logits


lstm_model = LstmModel(len(char_set))
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=5,
    verbose=True,
    mode="min" 
)
trainer = Trainer(
    gpus=0,
    checkpoint_callback=False,
    accumulate_grad_batches=1,
    max_epochs=200,
    progress_bar_refresh_rate=10,
    callbacks=[early_stop_callback])
trainer.fit(lstm_model, train_loader, val_loader)
trainer.test(lstm_model, test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name             | Type             | Params
------------------------------------------------------
0 | loss             | CrossEntropyLoss | 0     
1 | valid_accuracy   | Accuracy         | 0     
2 | test_accuracy    | Accuracy         | 0     
3 | embeddings_layer | Embedding        | 696   
4 | dropout          | Dropout          | 0     
5 | lstm_layer       | LSTM             | 6.6 K 
6 | out_layer        | Linear           | 27    
------------------------------------------------------
7.3 K     Trainable params
0         Non-trainable params
7.3 K     Total params
0.029     Total estimated model params size (MB)


Epoch 0: 100%|████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:04<00:00,  1.49it/s]
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                                             | 0/1 [00:00<?, ?it/s]

Metric val_loss improved. New best score: 1.170


Epoch 1: 100%|██████████████████████████████| 6/6 [00:04<00:00,  1.39it/s, loss=1.23, v_num=21, val_loss=1.170, val_acc_epoch=0.158]
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                                             | 0/1 [00:00<?, ?it/s]

Metric val_loss improved by 0.096 >= min_delta = 0.0. New best score: 1.074


...

Metric val_loss improved by 0.001 >= min_delta = 0.0. New best score: 0.665


Epoch 198: 100%|███████████████████████████| 6/6 [00:04<00:00,  1.48it/s, loss=0.684, v_num=21, val_loss=0.665, val_acc_epoch=0.712]
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                                             | 0/1 [00:00<?, ?it/s]

Metric val_loss improved by 0.001 >= min_delta = 0.0. New best score: 0.664


Epoch 199: 100%|███████████████████████████| 6/6 [00:04<00:00,  1.42it/s, loss=0.686, v_num=21, val_loss=0.664, val_acc_epoch=0.713]
Validating: 0it [00:00, ?it/s]
Testing: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.27it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.7302069067955017,
 'test_acc_epoch': 0.7302069067955017,
 'test_loss': 0.6225737929344177}
--------------------------------------------------------------------------------


[{'test_loss': 0.6225737929344177,
  'test_acc': 0.7302069067955017,
  'test_acc_epoch': 0.7302069067955017}]

In [24]:
predict(lstm_model, test_loader)

1 Precision: 0.47035573122529645, 1 Recall: 0.46484375
Exact: 502, partial: 900, missing: 646, spurius: 439
PREDICTED:


## Контекстная модель: LSTM над CharFF

In [25]:
import torch
from torch import nn
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping

class CharFFLstmModel(TemplateModel):
    def __init__(self, char_set_size, char_embedding_dim=4, classes_count=3,
                 word_embedding_dim=16, lstm_embedding_dim=16, char_max_seq_len=50):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(char_embedding_dim * char_max_seq_len, word_embedding_dim)
        self.relu = nn.ReLU()
        self.lstm_layer = nn.LSTM(word_embedding_dim, lstm_embedding_dim // 2, batch_first=True, bidirectional=True)
        self.out_layer = nn.Linear(lstm_embedding_dim, classes_count)

    def forward(self, inputs, labels):
        projections = self.embeddings_layer.forward(inputs)
        projections = projections.reshape(projections.size(0), projections.size(1), -1)
        projections = self.relu(self.linear(projections))
        projections = self.dropout(projections)
        output, _= self.lstm_layer(projections)
        output = self.dropout(output)
        logits = self.out_layer.forward(output)
        logits = logits.transpose(1, 2)
        loss = self.loss(logits, labels)
        return loss, logits


char_ff_lstm_model = CharFFLstmModel(len(char_set))
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=8,
    verbose=True,
    mode="min" 
)
trainer = Trainer(
    gpus=0,
    checkpoint_callback=False,
    accumulate_grad_batches=1,
    max_epochs=150,
    progress_bar_refresh_rate=10,
    callbacks=[early_stop_callback])
trainer.fit(char_ff_lstm_model, train_loader, val_loader)
trainer.test(char_ff_lstm_model, test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name             | Type             | Params
------------------------------------------------------
0 | loss             | CrossEntropyLoss | 0     
1 | valid_accuracy   | Accuracy         | 0     
2 | test_accuracy    | Accuracy         | 0     
3 | embeddings_layer | Embedding        | 696   
4 | dropout          | Dropout          | 0     
5 | linear           | Linear           | 3.2 K 
6 | relu             | ReLU             | 0     
7 | lstm_layer       | LSTM             | 1.7 K 
8 | out_layer        | Linear           | 51    
------------------------------------------------------
5.6 K     Trainable params
0         Non-trainable params
5.6 K     Total params
0.023     Total estimated model params size (MB)


Epoch 0: 100%|████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:04<00:00,  1.49it/s]
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                                             | 0/1 [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.981


Epoch 1: 100%|██████████████████████████████| 6/6 [00:03<00:00,  1.55it/s, loss=1.04, v_num=22, val_loss=0.981, val_acc_epoch=0.653]
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                                             | 0/1 [00:00<?, ?it/s]

Metric val_loss improved by 0.034 >= min_delta = 0.0. New best score: 0.947


...

Metric val_loss improved by 0.001 >= min_delta = 0.0. New best score: 0.691


Epoch 144: 100%|███████████████████████████| 6/6 [00:03<00:00,  1.59it/s, loss=0.688, v_num=22, val_loss=0.691, val_acc_epoch=0.700]
Validating: 0it [00:00, ?it/s]
Epoch 145: 100%|███████████████████████████| 6/6 [00:03<00:00,  1.61it/s, loss=0.695, v_num=22, val_loss=0.695, val_acc_epoch=0.702]
Validating: 0it [00:00, ?it/s]
Epoch 146: 100%|███████████████████████████| 6/6 [00:03<00:00,  1.61it/s, loss=0.695, v_num=22, val_loss=0.700, val_acc_epoch=0.702]
Validating: 0it [00:00, ?it/s]
Epoch 147: 100%|███████████████████████████| 6/6 [00:03<00:00,  1.60it/s, loss=0.689, v_num=22, val_loss=0.693, val_acc_epoch=0.699]
Validating: 0it [00:00, ?it/s]
Epoch 148: 100%|███████████████████████████| 6/6 [00:03<00:00,  1.58it/s, loss=0.688, v_num=22, val_loss=0.692, val_acc_epoch=0.703]
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                                             | 0/1 [00:00<?, ?it/s]

Metric val_loss improved by 0.002 >= min_delta = 0.0. New best score: 0.689


Epoch 149: 100%|███████████████████████████| 6/6 [00:03<00:00,  1.63it/s, loss=0.697, v_num=22, val_loss=0.689, val_acc_epoch=0.703]
Validating: 0it [00:00, ?it/s]
Testing: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 17.48it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.7229655385017395,
 'test_acc_epoch': 0.7229655385017395,
 'test_loss': 0.6334162354469299}
--------------------------------------------------------------------------------


[{'test_loss': 0.6334162354469299,
  'test_acc': 0.7229655385017395,
  'test_acc_epoch': 0.7229655385017395}]

In [26]:
predict(char_ff_lstm_model, test_loader)

1 Precision: 0.4488734835355286, 1 Recall: 0.37939453125
Exact: 425, partial: 901, missing: 722, spurius: 362
PREDICTED:


## Задание 1.1
Сделайте то же самое, но с bidirectional LSTM на уровне символов

In [27]:
import torch
from torch import nn
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping

class CharLstmModel(TemplateModel):
    def __init__(self, char_set_size, char_embedding_dim=4, classes_count=3,
                 lstm_embedding_dim=16, char_max_seq_len=50):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim)
        self.dropout = nn.Dropout(0.3)
        self.lstm_layer = nn.LSTM(char_embedding_dim, lstm_embedding_dim // 2, batch_first=True, bidirectional=True)
        self.out_layer = nn.Linear(lstm_embedding_dim * char_max_seq_len, classes_count)

    def forward(self, inputs, labels):
        projections = self.embeddings_layer.forward(inputs)
        char_max_seq_len = projections.size(2)
        projections = projections.reshape(projections.size(0), -1, projections.size(3))
        output, _= self.lstm_layer(projections)
        output = self.dropout(output)
        output = output.reshape(output.size(0), -1, output.size(2)*char_max_seq_len)
        logits = self.out_layer.forward(output)
        logits = logits.transpose(1, 2)
        loss = self.loss(logits, labels)
        return loss, logits


char_lstm_model = CharLstmModel(len(char_set))
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=8,
    verbose=True,
    mode="min" 
)
trainer = Trainer(
    gpus=0,
    checkpoint_callback=False,
    accumulate_grad_batches=1,
    max_epochs=150,
    progress_bar_refresh_rate=10,
    callbacks=[early_stop_callback])
trainer.fit(char_lstm_model, train_loader, val_loader)
trainer.test(char_lstm_model, test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name             | Type             | Params
------------------------------------------------------
0 | loss             | CrossEntropyLoss | 0     
1 | valid_accuracy   | Accuracy         | 0     
2 | test_accuracy    | Accuracy         | 0     
3 | embeddings_layer | Embedding        | 696   
4 | dropout          | Dropout          | 0     
5 | lstm_layer       | LSTM             | 896   
6 | out_layer        | Linear           | 2.4 K 
------------------------------------------------------
4.0 K     Trainable params
0         Non-trainable params
4.0 K     Total params
0.016     Total estimated model params size (MB)


Epoch 0: 100%|████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:52<00:00,  8.79s/it]
Validating: 0it [00:00, ?it/s]
Validating: 100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.90s/it]

Metric val_loss improved. New best score: 0.867


Epoch 1: 100%|█████████████████████████████| 6/6 [00:55<00:00,  9.26s/it, loss=0.981, v_num=23, val_loss=0.867, val_acc_epoch=0.653]
Validating: 0it [00:00, ?it/s]
Epoch 2: 100%|█████████████████████████████| 6/6 [00:55<00:00,  9.21s/it, loss=0.924, v_num=23, val_loss=0.877, val_acc_epoch=0.653]
Validating: 0it [00:00, ?it/s]
Validating: 100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]

Metric val_loss improved by 0.020 >= min_delta = 0.0. New best score: 0.848


...

Metric val_loss improved by 0.001 >= min_delta = 0.0. New best score: 0.701


Epoch 144: 100%|███████████████████████████| 6/6 [01:05<00:00, 10.88s/it, loss=0.704, v_num=23, val_loss=0.701, val_acc_epoch=0.687]
Validating: 0it [00:00, ?it/s]
Epoch 145: 100%|███████████████████████████| 6/6 [01:02<00:00, 10.49s/it, loss=0.696, v_num=23, val_loss=0.707, val_acc_epoch=0.675]
Validating: 0it [00:00, ?it/s]
Validating: 100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.29s/it]

Metric val_loss improved by 0.001 >= min_delta = 0.0. New best score: 0.700


Epoch 146: 100%|███████████████████████████| 6/6 [00:59<00:00,  9.85s/it, loss=0.688, v_num=23, val_loss=0.700, val_acc_epoch=0.687]
Validating: 0it [00:00, ?it/s]
Epoch 147: 100%|███████████████████████████| 6/6 [00:56<00:00,  9.47s/it, loss=0.692, v_num=23, val_loss=0.702, val_acc_epoch=0.686]
Validating: 0it [00:00, ?it/s]
Epoch 148: 100%|███████████████████████████| 6/6 [00:55<00:00,  9.33s/it, loss=0.698, v_num=23, val_loss=0.706, val_acc_epoch=0.682]
Validating: 0it [00:00, ?it/s]
Epoch 149: 100%|████████████████████████████| 6/6 [00:58<00:00,  9.68s/it, loss=0.69, v_num=23, val_loss=0.710, val_acc_epoch=0.681]
Validating: 0it [00:00, ?it/s]
Testing: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.49s/it]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.7013103365898132,
 'test_acc_epoch': 0.7013103365898132,
 'test_loss': 0.6508675217628479

[{'test_loss': 0.6508675217628479,
  'test_acc': 0.7013103365898132,
  'test_acc_epoch': 0.7013103365898132}]

In [28]:
predict(char_lstm_model, test_loader)

1 Precision: 0.42778119237861095, 1 Recall: 0.33984375
Exact: 396, partial: 762, missing: 890, spurius: 369
PREDICTED:


## Задание 1.2
Сделайте то же самое, но со свёртками на уровне символов

In [42]:
import torch
import torch.nn as nn
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping  

class CharCNNModel(TemplateModel):
    def __init__(self, char_set_size, char_embedding_dim=4, classes_count=3,
                 kernel_size=5, channels=500, char_max_seq_len=50):
        super().__init__()
        
        self.embedding = nn.Embedding(char_set_size, char_embedding_dim)
        self.cnn = nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=char_embedding_dim*kernel_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.out_layer = nn.Linear(char_embedding_dim*char_max_seq_len - (char_embedding_dim*kernel_size - 1), classes_count)
    
    def forward(self, inputs, labels):
        projections = self.embedding.forward(inputs)
        projections = projections.reshape(projections.size(0), projections.size(1), -1)
        convolved = self.dropout(self.relu(self.cnn(projections)))
        logits = self.out_layer.forward(convolved)
        
        logits = logits.transpose(1, 2)
        loss = self.loss(logits, labels)
        return loss, logits

char_cnn_model = CharCNNModel(len(char_set))
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=8,
    verbose=True,
    mode="min" 
)
trainer = Trainer(
    gpus=0,
    checkpoint_callback=False,
    accumulate_grad_batches=1,
    max_epochs=150,
    progress_bar_refresh_rate=10,
    callbacks=[early_stop_callback])
trainer.fit(char_cnn_model, train_loader, val_loader)
trainer.test(char_cnn_model, test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name           | Type             | Params
----------------------------------------------------
0 | loss           | CrossEntropyLoss | 0     
1 | valid_accuracy | Accuracy         | 0     
2 | test_accuracy  | Accuracy         | 0     
3 | embedding      | Embedding        | 696   
4 | cnn            | Conv1d           | 5.0 M 
5 | relu           | ReLU             | 0     
6 | dropout        | Dropout          | 0     
7 | out_layer      | Linear           | 546   
----------------------------------------------------
5.0 M     Trainable params
0         Non-trainable params
5.0 M     Total params
20.007    Total estimated model params size (MB)


Epoch 0: 100%|████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:11<00:00,  1.99s/it]
Validating: 0it [00:00, ?it/s]
Validating: 100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.47it/s]

Metric val_loss improved. New best score: 1.360


Epoch 1: 100%|███████████████████████████████| 6/6 [00:10<00:00,  1.81s/it, loss=2.2, v_num=24, val_loss=1.360, val_acc_epoch=0.625]
Validating: 0it [00:00, ?it/s]
Validating: 100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.09it/s]

Metric val_loss improved by 0.255 >= min_delta = 0.0. New best score: 1.105


...

Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.938


Epoch 7: 100%|█████████████████████████████| 6/6 [00:11<00:00,  1.91s/it, loss=0.964, v_num=24, val_loss=0.938, val_acc_epoch=0.637]
Validating: 0it [00:00, ?it/s]
Epoch 8: 100%|██████████████████████████████| 6/6 [00:12<00:00,  2.03s/it, loss=0.85, v_num=24, val_loss=0.939, val_acc_epoch=0.632]
Validating: 0it [00:00, ?it/s]
Epoch 9: 100%|█████████████████████████████| 6/6 [00:12<00:00,  2.07s/it, loss=0.762, v_num=24, val_loss=0.958, val_acc_epoch=0.623]
Validating: 0it [00:00, ?it/s]
Epoch 10: 100%|████████████████████████████| 6/6 [00:12<00:00,  2.08s/it, loss=0.679, v_num=24, val_loss=0.965, val_acc_epoch=0.616]
Validating: 0it [00:00, ?it/s]
Epoch 11: 100%|████████████████████████████| 6/6 [00:12<00:00,  2.15s/it, loss=0.607, v_num=24, val_loss=0.994, val_acc_epoch=0.607]
Validating: 0it [00:00, ?it/s]
Epoch 12: 100%|████████████████████████████| 6/6 [00:13<00:00,  2.18s/it, loss=0.552, v_num=24, val_loss=1.020, val_acc_epoch=0.601]
Validating: 0it [00:00, ?it/s]
Epoch 13: 100%|█

Monitored metric val_loss did not improve in the last 8 records. Best score: 0.938. Signaling Trainer to stop.


Testing: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.53it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.6251034736633301,
 'test_acc_epoch': 0.6251034736633301,
 'test_loss': 1.041354775428772}
--------------------------------------------------------------------------------


[{'test_loss': 1.041354775428772,
  'test_acc': 0.6251034736633301,
  'test_acc_epoch': 0.6251034736633301}]

In [43]:
predict(char_cnn_model, test_loader)

1 Precision: 0.2627226463104326, 1 Recall: 0.20166015625
Exact: 212, partial: 574, missing: 1262, spurius: 720
PREDICTED:
